<a href="https://colab.research.google.com/github/jaimebaldeon/Financial-Stock-Prediction/blob/yfinanceAPI/financialStocks_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prueba investpy
!pip install git+https://github.com/alvarobartt/investpy.git@developer
import investpy


In [ ]:

df = investpy.get_stock_historical_data(stock='BABA',
                                        country='United States',
                                        from_date='01/01/2010',
                                        to_date='01/01/2020')
print(df)

In [ ]:
data = investpy.get_crypto_historical_data(crypto='bitcoin', from_date='01/01/2014', to_date='01/01/2019')
print(data)

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as ur

In [ ]:
def convert_to_int(df): 
  col = df.columns
  for col in df: 
    temp = df[col].to_string().replace(',','').split('\n') #     Replace comma with space and clean out extra spaces
    temp.pop(0) #Remove first erronous element
    df.index = pd.to_datetime(df.index) # Convert index to datetime 
    df[col]= [i[10:].strip() for i in temp] # stripping off nth characters
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int) # Transform columns from   object to numeric
  return df
  
def clean_data(is_data):
  Fin_st = pd.DataFrame(is_data[0:])
  Fin_st.columns = Fin_st.iloc[0] # Name columns to first row of dataframe
  Fin_st = Fin_st.iloc[1:,] # start to read 1st row
  Fin_st = Fin_st.T # transpose dataframe
  Fin_st.columns = Fin_st.iloc[0] #Name columns to first row of dataframe
  Fin_st.drop(Fin_st.index[0],inplace=True) #Drop first index row
  Fin_st.index.name = '' # Remove the index name
  Fin_st.rename(index={'ttm': '12/31/2020'},inplace=True) #Rename ttm in index columns to end of the year
  #Fin_st = Fin_st[Fin_st.columns[:-5]] # remove last 5 irrelevant columns
  return Fin_st

def extract_data(url, fin_st):
  read_data = ur.urlopen(url).read()
  soup_is = BeautifulSoup(read_data, 'lxml')
  #print(soup_is)
  div = []
  for l in soup_is.find_all('div'):
    # Find all divs in site
    #print(l.string)
    if l.string is not None:
      div.append(str(l.string))
  div = list(filter(None, div))
  #print(div[15:])
  div = div[15:]
  # Add missing values in div
  div[0] = 'Annual'
  if fin_st is 'is':
    div = div[:6] + ['Total Revenue'] + div[6:]
    div = div[:24] + ['Operating Expense'] + div[24:]
    div = div[:36] + ['Net Non Operating Interest Income Expense'] + div[36:]
    div = div[:42] + ['Other Income Expense'] + div[42:]
    div = div[:60] + ['Net Income Common Stockholders'] + div[60:]
  elif fin_st is 'bs':
    div = div[:5] + ['Total Assets'] + div[5:]
    div = div[:10] + ['Total Liabilities Net Minority Interest'] + div[10:]
    div = div[:15] + ['Total Equity Gross Minority Interest'] + div[15:]
  elif fin_st is 'cf':
    div = div[:6] + ['Operating Cash Flow'] + div[6:]
    div = div[:12] + ['Investing Cash Flow'] + div[12:]
    div = div[:18] + ['Financing Cash Flow'] + div[18:]
    div = div[:24] + ['End Cash Position'] + div[24:]
  else:
    return False
  #print(new_ls)
  return div

In [ ]:
# Stock symbol
index= 'MOMO'

# URL link 
url_is = 'https://finance.yahoo.com/quote/' + index + '/financials?p=' + index
url_bs = 'https://finance.yahoo.com/quote/' + index +'/balance-sheet?p=' + index
url_cf = 'https://finance.yahoo.com/quote/' + index + '/cash-flow?p='+ index
print(url_is)


https://finance.yahoo.com/quote/MOMO/financials?p=MOMO


In [ ]:
##  INCOME STATEMENT

new_ls = extract_data(url_is, 'is')
is_data = list(zip(*[iter(new_ls)]*6))
Income_st = clean_data(is_data)
Income_st = convert_to_int(Income_st)
Income_st

Annual,Total Revenue,Cost of Revenue,Gross Profit,Operating Expense,Operating Income,Net Non Operating Interest Income Expense,Other Income Expense,Pretax Income,Tax Provision,Net Income Common Stockholders,"24,102",Diluted NI Available to Com Stockholders,Basic EPS,Diluted EPS,Basic Average Shares,Diluted Average Shares,Total Operating Income as Reported,Total Expenses,Net Income from Continuing & Discontinued Operation,Normalized Income,Interest Income,Interest Expense,Net Interest Income,EBIT,EBITDA,Reconciled Cost of Revenue,Reconciled Depreciation,Net Income from Continuing Operation Net Minority Interest,Total Unusual Items Excluding Goodwill,Total Unusual Items,Normalized EBITDA,Tax Rate for Calcs,Tax Effect of Unusual Items
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-12-31,16886254,8495438,8390816,4626470,3764346,365146,-17211,4112281,883041,0,3220479,3220479,0,0,0,0,3764346,13121908,3220479,3233994,444587,79441,365146,4191722,4553181,8495438,361459,3220479,-17211,-17211,4570392,0,-3696
2019-12-31,17015089,8492096,8522993,4968294,3554699,328931,-15711,3867919,883801,-17319,2970890,2970890,0,0,207658,225603,3554699,13460390,2970890,2983011,407542,78611,328931,3946530,0,8492096,356191,2970890,-15711,-15711,4318432,0,-3590
2018-12-31,13408421,7182897,6225524,2959232,3266292,216443,-43200,3439535,699648,-23350,2815775,2815775,0,0,203505,216542,3266292,10142129,2815775,2850188,272946,56503,216443,3496038,0,7182897,241268,2815775,-43200,-43200,3780506,0,-8787
2017-12-31,8606583,4238915,4367667,2011685,2355982,141248,-28635,2468596,430763,48660,2079819,2079819,0,0,197275,207633,2355982,6250600,2079819,2103457,141248,0,141248,2355982,0,4238915,81191,2079819,-28635,-28635,2465808,0,-4997
2016-12-31,3840602,1676671,2163931,1160649,1003282,56897,-40031,1020148,35663,38447,986388,986388,0,0,188668,203521,1003282,2837320,1008587,1047219,56897,0,56897,1003282,0,1676671,58356,1008587,-40031,-40031,1101669,0,-1399


In [ ]:
##  BALANCE SHEET

new_ls = extract_data(url_bs, 'bs')
is_data = list(zip(*[iter(new_ls)]*5))
Balance_sh = clean_data(is_data)
Balance_sh = convert_to_int(Balance_sh)
#Balance_sh

In [ ]:
##  CASH FLOW

new_ls = extract_data(url_cf, 'cf')
is_data = list(zip(*[iter(new_ls)]*6))
CashFlow_st = clean_data(is_data)
CashFlow_st = convert_to_int(CashFlow_st)
CashFlow_st

Annual,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,End Cash Position,Capital Expenditure,Issuance of Capital Stock,Issuance of Debt,Repayment of Debt,Repurchase of Capital Stock,Free Cash Flow
,,,,,,,,,,
2020-12-31,4816473,-3739444,-968702,2786669,-171891,0,0,0,0,4644582
2019-12-31,5448886,-4029919,-1273780,2612743,-186522,0,0,0,0,5262364
2018-12-31,3327718,-10034004,4687951,2468034,-242843,0,6732868,-2041680,0,3084875
2017-12-31,2791714,-178299,2742,4477552,-230508,0,0,0,0,2561205
2016-12-31,1515908,-821785,97,1788473,-48787,0,0,0,0,1467121


In [ ]:
# EBIT
EBIT = Income_st['Operating Income']
print(EBIT)
# EBITDA
#print(Income_st[['Operating Income', "Reconciled Depreciation"]])
EBITDA = EBIT + Income_st["Reconciled Depreciation"] # Income_st["EBITDA"] 
print(EBITDA)

# FCF (Operating Cash Flow - |Investing Cash Flow (CAPEX)|)
# Find LFCF

# Otherwise, compute FCF
#print(CashFlow_st[['Operating Cash Flow','Investing Cash Flow'] ])
FCF = CashFlow_st['Operating Cash Flow'] + CashFlow_st['Investing Cash Flow'] 
print(FCF)

# ENTERPRISE VALUE (Market Cap - Short-Term Debt + L-T Debt - Cash And Cash Equivalents)


# EV/EBIT

# EV/EBITDA

# EV/FCF

# PER



2020-12-31    3764346
2019-12-31    3554699
2018-12-31    3266292
2017-12-31    2355982
2016-12-31    1003282
Name: Operating Income, dtype: int64

2020-12-31    4125805
2019-12-31    3910890
2018-12-31    3507560
2017-12-31    2437173
2016-12-31    1061638
dtype: int64

2020-12-31    1077029
2019-12-31    1418967
2018-12-31   -6706286
2017-12-31    2613415
2016-12-31     694123
dtype: int64


In [ ]:
!pip install yfinance

In [22]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get stock info
msft.info

# get historical market data
hist = msft.history(period="max")

# show actions (dividends, splits)
msft.actions

# show dividends
msft.dividends

# show splits
msft.splits

# show financials
msft.financials
msft.quarterly_financials

# show major holders
msft.major_holders

# show institutional holders
msft.institutional_holders

# show balance sheet
msft.balance_sheet
msft.quarterly_balance_sheet

# show cashflow
msft.cashflow
msft.quarterly_cashflow

# show earnings
msft.earnings
msft.quarterly_earnings

# show sustainability
msft.sustainability

# show analysts recommendations
msft.recommendations

# show next event (earnings, etc)
msft.calendar

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
msft.isin

# show options expirations
msft.options

# get option chain for specific expiration
#opt = msft.option_chain('YYYY-MM-DD')
msft.balance_sheet

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [8]:
# EBIT
# EBIT = Income_st['Operating Income']
# EBITDA
#print(Income_st[['Operating Income', "Reconciled Depreciation"]])
#EBITDA = EBIT + Income_st["Reconciled Depreciation"] # Income_st["EBITDA"] 

# FCF (Operating Cash Flow - |Investing Cash Flow (CAPEX)|)
# Find LFCF

# Otherwise, compute FCF
#print(CashFlow_st[['Operating Cash Flow','Investing Cash Flow'] ])
#FCF = CashFlow_st['Operating Cash Flow'] + CashFlow_st['Investing Cash Flow'] 

# ENTERPRISE VALUE (Market Cap - Short-Term Debt + L-T Debt - Cash And Cash Equivalents)
M_Cap = msft.info['marketCap']
ST_Debt = msft.info['']
print(M_Cap)
# EV/EBIT

# EV/EBITDA

# EV/FCF

# PER
PER = msft.info['trailingPE']
print(PER)

1598213324800
34.100643
